In [3]:
"""


Original paper - https://arxiv.org/abs/1611.08024



"""

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

In [13]:
# load the dataframe from the pickle file
import pickle
dir = "C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData"
with open(f"{dir}/dataframe.pkl", "rb") as f:
    df = pickle.load(f)

# keep these channels only, these are the indexes: 
""" 
Fp1 -> 0
Fp2 -> 33
F3  -> 4
F4  -> 38
C3  -> 12
C4  -> 48
P3  -> 20
P4  -> 55
O1  -> 26
O2  -> 61
F7  -> 6
F8  -> 40
T7  -> 14
T8  -> 50
P7  -> 22
P8  -> 57
Fz  -> 36
Cz  -> 46
Pz  -> 30 

but add 1 to each index, since the first channel is channel_1
"""

df = df[["channel_1", "channel_34", "channel_5", "channel_39", "channel_13", "channel_49", "channel_21", "channel_56", "channel_27", "channel_62", "channel_7", "channel_41", "channel_15", "channel_51", "channel_23", "channel_58", "channel_37", "channel_47", "channel_31", "label"]]

patient_ids = np.repeat([1,2,3,4,5,6,7,8,9,10],76288)  # Make sure to have this aligned with your epochs/labels

# Normalize per patient (within training and test sets)
data = df.drop("label", axis=1).values
data_norm = []
for patient_id in np.unique(patient_ids):
    patient_data = data[patient_ids == patient_id]
    scaler = StandardScaler()
    patient_data_scaled = scaler.fit_transform(patient_data)
    print(f"mean of patient {patient_id}: {np.min(patient_data_scaled), np.max(patient_data_scaled)}")
    data_norm.append(patient_data_scaled)

data_norm = np.concatenate(data_norm, axis=0)
# add labels back
y = df["label"].values
data = data_norm

# make data a dataframe again
df = pd.DataFrame(data)
print(f"size of df: {df.shape}")

# split the data into epochs of 256 datapoints each
epochs = []
for i in range(0, len(df), 256):
    epochs.append(df.iloc[i:i+256].values)

# convert the list of epochs to a numpy array
epochs = np.array(epochs)
print(f"size of epochs: {epochs.shape}")

# save the SCALED epochs
with open(f"{dir}/scaled_rawEEG_epochs.pkl", "wb") as f:
    pickle.dump(epochs, f)




mean of patient 1: (np.float64(-2.5061664365916747), np.float64(3.2048110636610154))
mean of patient 2: (np.float64(-2.2856199344616766), np.float64(3.1704214250154332))
mean of patient 3: (np.float64(-2.6244058798465795), np.float64(2.326236025125405))
mean of patient 4: (np.float64(-2.9839669159521462), np.float64(3.0033738566092985))
mean of patient 5: (np.float64(-2.7180645928577865), np.float64(1.9617189652654394))
mean of patient 6: (np.float64(-3.8235823428136357), np.float64(1.5538075585523163))
mean of patient 7: (np.float64(-2.17836904765987), np.float64(2.700097334829556))
mean of patient 8: (np.float64(-1.9391087092804788), np.float64(4.868952462591622))
mean of patient 9: (np.float64(-3.278040789072896), np.float64(2.18437991679923))
mean of patient 10: (np.float64(-1.6112388744411374), np.float64(4.696875071895533))
size of df: (762880, 19)
size of epochs: (2980, 256, 19)


In [2]:
labels = []
for _ in range(10):
    labels.append(np.repeat([1, 0], 149))
labels = np.concatenate(labels)

print(f"size of labels: {labels.shape}")


size of labels: (2980,)


In [3]:
channels = 19
sample_len = 256


class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = sample_len
        self.dropout_rate = 0.1
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, channels), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(8*2 * (sample_len // 32), 1)
        

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, self.dropout_rate)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, self.dropout_rate)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, self.dropout_rate)
        x = self.pooling3(x)
        
        # FC Layer
        x = x.reshape(-1, 128)
        x = F.sigmoid(self.fc1(x))
        return x


net = EEGNet()
print(net.forward(Variable(torch.Tensor(1,1,sample_len,channels))))
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)

net.apply(init_weights)


tensor([[0.5675]], grad_fn=<SigmoidBackward0>)


EEGNet(
  (conv1): Conv2d(1, 16, kernel_size=(1, 19), stride=(1, 1))
  (batchnorm1): BatchNorm2d(16, eps=False, momentum=0.1, affine=True, track_running_stats=True)
  (padding1): ZeroPad2d((16, 17, 0, 1))
  (conv2): Conv2d(1, 4, kernel_size=(2, 32), stride=(1, 1))
  (batchnorm2): BatchNorm2d(4, eps=False, momentum=0.1, affine=True, track_running_stats=True)
  (pooling2): MaxPool2d(kernel_size=2, stride=4, padding=0, dilation=1, ceil_mode=False)
  (padding2): ZeroPad2d((2, 1, 4, 3))
  (conv3): Conv2d(4, 4, kernel_size=(8, 4), stride=(1, 1))
  (batchnorm3): BatchNorm2d(4, eps=False, momentum=0.1, affine=True, track_running_stats=True)
  (pooling3): MaxPool2d(kernel_size=(2, 4), stride=(2, 4), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=1, bias=True)
)

# torcheeg trainer

In [58]:
from torch.utils.data import DataLoader
from torcheeg.models import CCNN
from torcheeg.trainers import ClassifierTrainer
import pytorch_lightning as pl

from sklearn.model_selection import LeaveOneGroupOut
batch_size = 16


# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# This is an array where each entry corresponds to a patient ID (e.g., [0, 0, 0, 1, 1, 1, ..., 9])
patient_ids = np.repeat([1,2,3,4,5,6,7,8,9,10],298)  # Make sure to have this aligned with your epochs/labels


for i, (train_index, test_index) in enumerate(logo.split(epochs, labels, groups=patient_ids)):
        # Training loop

    # Train and test split
    X_train, X_val = epochs[train_index,:,:], epochs[test_index,:,:]
    y_train, y_val = labels[train_index], labels[test_index]
    
    # Create DataLoader for training and testing
    train_dataset = EEGDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_dataset = EEGDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Make sure that when the batches are concatenated, the first dimension is the channel dimension, then the batch_size


    model = CCNN(num_classes=2, in_channels=19, grid_size=(5,5))

    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=1e-4,
                                weight_decay=1e-4,
                                accelerator="cpu")
    trainer.fit(train_loader,
                val_loader,
                max_epochs=50,
                default_root_dir=f'C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData/{i}',
                callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
                enable_progress_bar=True,
                enable_model_summary=True,
                limit_val_batches=0.0)
    
    score = trainer.test(val_loader,
                        enable_progress_bar=True,
                        enable_model_summary=True)[0]
    print(f'Fold {i} test accuracy: {score["test_accuracy"]:.4f}')



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | CCNN             | 17.7 M | train
1 | ce_fn         | CrossEntropyLoss | 0      | train
2 | train_loss    | MeanMetric       | 0      | train
3 | val_loss      | MeanMetric       | 0      | train
4 | test_loss     | MeanMetric       | 0      | train
5 | train_metrics | MetricCollection | 0      | train
6 | val_metrics   | MetricCollection | 0      | train
7 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
17.7 M    Trainable params
0         Non-trainable params
17.7 M    Total params
70.871    Total estimated model params size (MB)
32        Modules in train mode
0         Modules in eval mode
c:\Users\gusta\miniconda3\envs\BACHELOR\Lib\site-packages\pytorch_lightning\trainer\conne

Epoch 0:   0%|          | 0/168 [00:00<?, ?it/s] 

RuntimeError: Given groups=1, weight of size [64, 19, 4, 4], expected input[1, 16, 22, 259] to have 19 channels, but got 16 channels instead

In [56]:
train_loader.dataset.__getitem__(0)
print(f"Shape of a batch from train_loader: {next(iter(train_loader))[0].shape}")


Shape of a batch from train_loader: torch.Size([16, 19, 256])


In [31]:
from torcheeg.models import EEGNet
from torch.utils.data import DataLoader
from torcheeg.model_selection import KFoldPerSubject


dataset = EEGDataset(epochs, labels)

# k_fold = KFoldPerSubject(n_splits=10, shuffle=True, random_state=42)


# dataset = EEGDataset(epochs, labels)


model = EEGNet(chunk_size=256,
               num_electrodes=19,
               dropout=0.5,
               kernel_1=64,
               kernel_2=16,
               F1=8,
               F2=16,
               D=2,
               num_classes=2)

# x, y = next(iter(DataLoader(dataset, batch_size=64)))
# model(x)

In [15]:
# open the scaled epochs
import pickle
import pandas as pd
dir = "C:/Users/gusta/OneDrive/Skrivebord/KI & Data/Bachelor/LegeData"

with open(f"{dir}/scaled_rawEEG_epochs.pkl", "rb") as f:
    epochs = pickle.load(f)




Evaluate function


In [26]:
import torch
from torch.autograd import Variable
import numpy as np
from sklearn.metrics import accuracy_score

# open the scaled epochs
with open(f"{dir}/scaled_rawEEG_epochs.pkl", "rb") as f:
    epochs = pickle.load(f)

# Define simple evaluation function
def evaluate(model, dataloader, sample_len, channels):
    model.eval()
    model_predictions = []
    true_labels = []

    for _, (data, target) in enumerate(dataloader):

      inputs, targets = data , target
      # permute inputs from (batch_size, channels, sample_len) to (batch_size, sample_len, channels)

      data = Variable((inputs))
      target = Variable((targets))
      
      output = model(data).squeeze()

      model_predictions.extend(output.detach().numpy())
      true_labels.extend(target.detach().numpy())

    all_predictions = np.array(model_predictions) > 0.5
    all_targets = np.array(true_labels)



    accuracy = accuracy_score(all_targets, all_predictions)
    return accuracy

In [27]:
from torch.utils.data import DataLoader, Dataset

# Dataset class definition
class EEGDataset(Dataset):
    def __init__(self, epochs, labels):
        self.df = epochs
        self.labels = labels

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Access the epoch and corresponding label
        data = self.df[idx]  # data shape: (256, 19)
        data = data.T  # shape: (19, 256)
        label = self.labels[idx]  # label shape: (1)
        
        # Ensure the data is converted to float tensor
        data_tensor = torch.tensor(data, dtype=torch.float32)
        
        # Convert label to tensor (assumed to be scalar)
        label_tensor = torch.tensor(label, dtype=torch.float32) 
        
        return data_tensor, label_tensor




# Homemade small CNN 2,033 parameters


In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EEGClassifier(nn.Module):
    def __init__(self, num_channels=19, num_timepoints=256):
        super(EEGClassifier, self).__init__()

        # 1st Conv Layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32))  # Conv over time
        self.bn1 = nn.BatchNorm2d(16)  # Batch normalization after convolution
        self.pool1 = nn.AvgPool2d(kernel_size=(1, 4))  # Pooling over time dimension
        self.drop1 = nn.Dropout(0.25)  # Dropout after first block

        # 2nd Conv Layer (Depthwise Separable Convolution)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(num_channels, 1), groups=16)  # Depthwise over channels
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.AvgPool2d(kernel_size=(1, 8))
        self.drop2 = nn.Dropout(0.25)

        # Fully Connected Layer for classification (sigmoid for binary classification)
        self.fc = nn.Linear(32 * (num_timepoints // 32), 1)  # Output 1 neuron for binary classification

    def forward(self, x):
        # Input x is (batch_size, num_channels, num_timepoints)
        x = x.unsqueeze(1)  # Add a channel dimension: (batch_size, 1, num_channels, num_timepoints)

        # 1st Conv Block
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = self.drop1(x)

        # 2nd Conv Block
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = self.drop2(x)

        # Flatten for fully connected layer
        x = x.view(x.size(0), -1)

        # Fully Connected Layer with sigmoid activation for binary classification
        x = torch.sigmoid(self.fc(x))
        return x

# Initialize the model
model = EEGClassifier(num_channels=19, num_timepoints=256)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# explore model size and trainable parameters
from torchsummary import summary
summary(model, (19, 256))




----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 19, 257]           1,040
       BatchNorm2d-2          [-1, 16, 19, 257]              32
         AvgPool2d-3           [-1, 16, 19, 64]               0
           Dropout-4           [-1, 16, 19, 64]               0
            Conv2d-5            [-1, 32, 1, 64]             640
       BatchNorm2d-6            [-1, 32, 1, 64]              64
         AvgPool2d-7             [-1, 32, 1, 8]               0
           Dropout-8             [-1, 32, 1, 8]               0
            Linear-9                    [-1, 1]             257
Total params: 2,033
Trainable params: 2,033
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 1.52
Params size (MB): 0.01
Estimated Total Size (MB): 1.55
-----------------------------------------------

# Slightly bigger CNN 204,737 parameters

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LargeEEGClassifier(nn.Module):
    def __init__(self, num_channels=19, num_timepoints=256):
        super(LargeEEGClassifier, self).__init__()

        # 1st Conv Layer: more filters and wider kernel
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32))  # Conv over time
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.AvgPool2d(kernel_size=(1, 4))
        self.drop1 = nn.Dropout(0.25)

        # 2nd Conv Layer: more filters
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(num_channels, 1), groups=64)  # Depthwise over channels
        self.bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.AvgPool2d(kernel_size=(1, 8))
        self.drop2 = nn.Dropout(0.25)

        # 3rd Conv Layer: additional layer to increase complexity
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(1, 4), padding=(0, 2))
        self.bn3 = nn.BatchNorm2d(256)
        self.pool3 = nn.AvgPool2d(kernel_size=(1, 4))
        self.drop3 = nn.Dropout(0.25)

        # Fully Connected Layer for classification
        self.fc1 = nn.Linear(256 * (num_timepoints // 128), 128)  # First fully connected layer
        self.fc2 = nn.Linear(128, 1)  # Second fully connected layer for binary classification

    def forward(self, x):
        # Input x is (batch_size, num_channels, num_timepoints)
        x = x.unsqueeze(1)  # Add a channel dimension: (batch_size, 1, num_channels, num_timepoints)

        # 1st Conv Block
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = self.drop1(x)

        # 2nd Conv Block
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = self.drop2(x)

        # 3rd Conv Block
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = self.drop3(x)

        # Flatten for fully connected layer
        x = x.view(x.size(0), -1)

        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid for binary classification
        return x

# Initialize the model
model = LargeEEGClassifier(num_channels=19, num_timepoints=256)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Explore model size and trainable parameters
summary(model, (19, 256))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 19, 257]           4,160
       BatchNorm2d-2          [-1, 64, 19, 257]             128
         AvgPool2d-3           [-1, 64, 19, 64]               0
           Dropout-4           [-1, 64, 19, 64]               0
            Conv2d-5           [-1, 128, 1, 64]           2,560
       BatchNorm2d-6           [-1, 128, 1, 64]             256
         AvgPool2d-7            [-1, 128, 1, 8]               0
           Dropout-8            [-1, 128, 1, 8]               0
            Conv2d-9            [-1, 256, 1, 9]         131,328
      BatchNorm2d-10            [-1, 256, 1, 9]             512
        AvgPool2d-11            [-1, 256, 1, 2]               0
          Dropout-12            [-1, 256, 1, 2]               0
           Linear-13                  [-1, 128]          65,664
           Linear-14                   

#### Training and evaluation loop

In [55]:
from sklearn.model_selection import LeaveOneGroupOut
from tqdm import tqdm

# define random seed
torch.manual_seed(42)


batch_size = 16
num_epochs = 50
patience = 20  # Stop if no improvement after 5 epochs
best_val_acc = 0.0  # Track the best validation accuracy
no_improvement_epochs = 0  # Track how many epochs without improvement

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Ensure patient_ids has the same length as epochs and labels
patient_ids = np.repeat([1,2,3,4,5,6,7,8,9,10], len(epochs) // 10)  # Adjust the length accordingly

num_epochs = 50

for epoch in tqdm(range(num_epochs)):  # Loop over the number of epochs
    print(f"Epoch {epoch+1}")
    running_loss = []

    for train_index, test_index in logo.split(epochs, labels, groups=patient_ids):
        # Training loop
        model.train()

        # Train and test split
        X_train, X_val = epochs[train_index,:,:], epochs[test_index,:,:]
        y_train, y_val = labels[train_index], labels[test_index]
        
        # Create DataLoader for training and testing
        train_dataset = EEGDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        val_dataset = EEGDataset(X_val, y_val)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        

        for i, data in enumerate(train_loader, 0):
            
            
            # Get the inputs; data is a list of [inputs, labels/targets]
            inputs, targets = data

            # Convert inputs and targets to variables
            inputs, targets = Variable(inputs), Variable(targets)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss.append(loss.item())


        # Check performance on training and testing sets # Evaluate model on validation set
    model.eval()
    with torch.no_grad():
        accuracy = evaluate(model, val_loader, sample_len, channels)
        val_loss = criterion(model(Variable(torch.Tensor(X_val))), Variable(torch.Tensor(y_val)))
        print(f'Epoch {epoch+1}, Validation Accuracy: {accuracy}')
        print(f"training loss: {np.mean(running_loss)}")

        # Early Stopping logic
    if accuracy > best_val_acc:
        best_val_acc = accuracy  # Update the best validation accuracy
        no_improvement_epochs = 0  # Reset the counter if there's improvement
        torch.save(model.state_dict(), f"{dir}/best_large_CNN_model.pth")  # Save the best model
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        break  # Exit the training loop
        

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1


  2%|▏         | 1/50 [01:31<1:14:35, 91.33s/it]

Epoch 1, Validation Accuracy: 0.9194630872483222
training loss: 0.5220033956248136
Epoch 2


  4%|▍         | 2/50 [03:12<1:17:30, 96.89s/it]

...

Epoch 31, Validation Accuracy: 0.9395973154362416
training loss: 0.09626270448484103
Epoch 32


 64%|██████▍   | 32/50 [51:03<28:13, 94.06s/it]

Epoch 32, Validation Accuracy: 0.9463087248322147
training loss: 0.09146018237155486
Epoch 33


 64%|██████▍   | 32/50 [52:43<29:39, 98.85s/it]

Epoch 33, Validation Accuracy: 1.0
training loss: 0.08937568076026288
Early stopping at epoch 33


In [39]:
# save the small model
torch.save(model.state_dict(), f"{dir}/smallCNNmodel.pth")

In [43]:
# save the large model
torch.save(model.state_dict(), f"{dir}/largeCNNmodel.pth")